In [ ]:
from graphviz import Digraph
dot = Digraph('infra', format='png')  
dot.attr(rankdir='LR', fontsize='10')  
  
# Global node style  
dot.node_attr.update(shape='box', style='filled', fillcolor='#E8F0FE', color='#4285F4', fontname='Helvetica', fontsize='10')  

# Legend as graph label at the bottom
dot.attr(labelloc='b', labeljust='c', label='''<
    <TABLE BORDER="0" CELLBORDER="0" CELLSPACING="8" CELLPADDING="4">
        <TR>
            <TD BGCOLOR="#FFCDD2" BORDER="1">  </TD><TD>Admin Services</TD>
            <TD BGCOLOR="#BBDEFB" BORDER="1">  </TD><TD>Core Services</TD>
            <TD BGCOLOR="#D1C4E9" BORDER="1">  </TD><TD>Auth Layer</TD>
            <TD BGCOLOR="#C8E6C9" BORDER="1">  </TD><TD>Identity Provider</TD>
        </TR>
    </TABLE>
>''')

# Create Hospital Intranet cluster  
with dot.subgraph(name='cluster_intranet') as intranet:  
    intranet.attr(label='Intranet', style='rounded', labelloc='t')  
  
    # Kubernetes cluster subgraph  
    with intranet.subgraph(name='cluster_k8s') as k8s:  
        k8s.attr(label='Kubernetes Cluster', style='rounded,dashed', labelloc='t')  
  
        # Admin services  
        with k8s.subgraph(name='cluster_admin') as admin:  
            admin.attr(label='Scout Admin Services', style='dotted', labelloc='t')  
            admin.node_attr.update(fillcolor='#FFCDD2', color='#C62828')  
            # Keycloak striped: purple (auth), red (admin), green (identity/SSO)
            admin.node('Keycloak', 'Keycloak', style='filled,striped', 
                       fillcolor='#D1C4E9:#FFCDD2:#C8E6C9', color='#333333')
            admin.node('Grafana', 'Grafana')  
            admin.node('Minio', 'MinIO')  
            admin.node('Temporal', 'Temporal')  
        
        # Core services  
        with k8s.subgraph(name='cluster_back') as back:  
            back.attr(label='Scout Core Services', style='dotted', labelloc='t')  
            back.node_attr.update(fillcolor='#BBDEFB', color='#1976D2')  
            back.node('SS', 'Superset')  
            back.node('JH', 'JupyterHub')
            back.node('Chat', 'Open WebUI')
            back.node('PB', 'Playbooks')
            back.node('Launchpad', 'Launchpad')
  
        # Utility services  
        with k8s.subgraph(name='cluster_util') as util:  
            util.attr(label='Authentication Layer', style='dotted', labelloc='t')  
            util.node_attr.update(fillcolor='#D1C4E9', color='#512DA8')  
            util.node('Ingress', 'Traefik Ingress')  
            util.node('OAuth2', 'OAuth2 Proxy\n200 → Logged In\n401 → Sign-in\n403 → Pending Approval')
  
    # User traffic (internal users)  
    intranet.node('Admins', 'Admins', fillcolor='#FFCDD2', color='#C62828')  
    intranet.node('Users', 'Clinicians / Researchers', fillcolor='#BBDEFB', color='#1976D2')  

    # Identity Provider
    with intranet.subgraph(name='cluster_idp') as idp:
        idp.attr(label='Identity Provider', style='dashed', labelloc='t')
        idp.node('SSO', 'Institutional SSO', fillcolor='#C8E6C9', color='#388E3C')

# Invisible edge to keep Identity Provider below Kubernetes cluster
dot.edge('Keycloak', 'SSO', style='invis')
    
# Routing  
dot.edge('Users', 'Ingress')  
dot.edge('Admins', 'Ingress')  
dot.edge('Ingress', 'OAuth2', dir='both', label='auth middleware')
dot.edge('Ingress', 'Minio')  
dot.edge('Ingress', 'Temporal')  
dot.edge('Ingress', 'Grafana')   
dot.edge('Ingress', 'Keycloak')
dot.edge('Ingress', 'SS', label='authenticated')  
dot.edge('Ingress', 'Launchpad')  
dot.edge('Ingress', 'JH')
dot.edge('Ingress', 'Chat')
dot.edge('Ingress', 'PB')
  
dot.edge('OAuth2', 'Keycloak', dir='both')
dot.edge('SSO', 'Keycloak', dir='both')
dot.edge('Launchpad', 'Keycloak', dir='both', style='dashed')
dot.edge('JH', 'Keycloak', dir='both', style='dashed')  
dot.edge('SS', 'Keycloak', dir='both', style='dashed')  
dot.edge('Minio', 'Keycloak', dir='both', style='dashed')  
dot.edge('Temporal', 'Keycloak', dir='both', style='dashed')
dot.edge('Grafana', 'Keycloak', dir='both', style='dashed')
dot.edge('Chat', 'Keycloak', dir='both', style='dashed')
dot.edge('PB', 'Keycloak', dir='both', style='dashed')

# Render  
file_path = 'scout_auth_diagram'  
dot.render(file_path, cleanup=True)  
  
file_path + '.png'  
dot